In [1]:
# Todd McCullough
# 2020
import numpy as np
import pandas as pd
import cpl_main as cpl 
import random
import re

In [3]:
import pickle
filename = 'models/cpl_MATCH_classifier-08-19-.sav'

cpl_classifier_model = pickle.load(open(filename, 'rb'))

regressor = 'models/cpl_score_regressor-07-20-vr---34.sav'
cpl_score_model = pickle.load(open(regressor, 'rb'))

In [4]:
year = '2020'
team_ref = pd.read_csv('datasets/teams.csv')
team_ref = team_ref[team_ref['year'] == int(year)]

In [5]:
results = pd.read_csv(f'datasets/{year}/cpl-{year}-results.csv')
results_old = pd.read_csv(f'datasets/{year}/cpl-{year}-results_old.csv')
stats = pd.read_csv(f'datasets/{year}/cpl-{year}-stats.csv')
stats_old = pd.read_csv(f'datasets/{year}/cpl-{year}-stats_old.csv')
stats_seed = pd.read_csv(f'datasets/{year}/cpl-{year}-stats-seed.csv')
player_info = pd.read_csv(f'datasets/{year}/player-{year}-info.csv')
#results_brief = pd.read_csv(f'datasets/{year}/cpl-{year}-results_brief.csv')
team_stats = pd.read_csv(f'datasets/{year}/cpl-{year}-team_stats.csv')
schedule = pd.read_csv(f'datasets/{year}/cpl-{year}-schedule.csv')
rated_forwards = pd.read_csv(f'datasets/{year}/cpl-{year}-forwards.csv')
rated_midfielders = pd.read_csv(f'datasets/{year}/cpl-{year}-midfielders.csv')
rated_defenders = pd.read_csv(f'datasets/{year}/cpl-{year}-defenders.csv')
rated_keepers = pd.read_csv(f'datasets/{year}/cpl-{year}-keepers.csv')

In [5]:
### not needed anymore
def get_team_comparison(data,q1,q2):
    # getting games with q1 in both home or away
    db = data[(data['home'] == q1) | (data['away'] == q1)]
    db = db.reset_index()
    db.pop('index')
    # filering down more to get only the games against q2
    db = db.sort_values(by=['m','d'],ascending=False)
    db = db[(db['home'] == q2) | (db['away'] == q2)]
    #print(db)
    db = db.reset_index()
    db.pop('index')
    if db.empty == True:
        db = pd.DataFrame([(0,0,0,0,q1,'D',q2,'D','empty',q1)],columns=['d','m','hs','as','home','hr','away','ar','summary','team'])
    if db.shape[0] > 15:
        return db.head(15)
    else:
        return db

In [6]:
t = 0

In [7]:
# home side
q1 = schedule.iloc[t]['home']
# away side
q2 = schedule.iloc[t]['away']
print(q1,q2)

Forge FC Cavalry FC


In [8]:
results_brief = cpl.get_results_brief(results,team_ref)
results_brief_old = cpl.get_results_brief(results_old,team_ref)
results_brief = pd.concat([results_brief,results_brief_old])
compare = cpl.get_team_comparison(results_brief,q1,q2)
compare = compare[compare['hr'] != 'E']
compare

,d,m,hs,as,home,hr,away,ar,summary,team
0,12,5,1.0,2.0,Forge FC,L,Cavalry FC,W,L H 1 - 2 CFC,Forge FC
1,22,6,0.0,1.0,Cavalry FC,L,Forge FC,W,W A 0 - 1 CFC,Forge FC
2,13,8,2.0,2.0,Forge FC,D,Cavalry FC,D,D H 2.0 - 2.0 CFC,Forge FC
3,25,8,1.0,0.0,Forge FC,W,Cavalry FC,L,W H 1 - 0 CFC,Forge FC
4,9,10,2.0,1.0,Cavalry FC,W,Forge FC,L,L A 2 - 1 CFC,Forge FC
5,16,10,1.0,0.0,Forge FC,W,Cavalry FC,L,W H 1 - 0 CFC,Forge FC
6,26,10,1.0,0.0,Forge FC,W,Cavalry FC,L,W H 1 - 0 CFC,Forge FC
7,2,11,0.0,1.0,Cavalry FC,L,Forge FC,W,W A 0 - 1 CFC,Forge FC


In [9]:
t1_x, t1_y = cpl.get_NB_data(compare,q1)
t2_x, t2_y = cpl.get_NB_data(compare,q2)

In [10]:
game_info = schedule[schedule['home'] == q1]
game_info = game_info[game_info['away'] == q2]
game = game_info.iloc[0]['game']
game_h = cpl.get_home_away_comparison(stats,game,q1)
game_a = cpl.get_home_away_comparison(stats,game,q2)

In [11]:
### FIX THE ERROR

# https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
### 

In [12]:
home_roster = cpl.best_roster(q1,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)
away_roster = cpl.best_roster(q2,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)

In [13]:
q1_roster = cpl.get_overall_roster(home_roster,player_info)
q2_roster = cpl.get_overall_roster(away_roster,player_info)

In [14]:
home_win, draw, away_win = cpl.get_nb_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
print(home_win, draw, away_win)

0.42 0.34 0.25


In [15]:
q1_roster_r = cpl.add_regressor_features(q1_roster)
q1_roster_r

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.71,0.71,0.74,0.9,0.77,0.5,0.52,0.45,0.68,0.42,...,3,4,4,4,4,2,4,5,4,1


In [16]:
q2_roster_r = cpl.add_regressor_features(q2_roster)
q2_roster_r

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.79,0.92,0.75,0.37,0.33,0.67,0.67,0.35,0.37,0.73,...,3,4,4,4,4,2,4,4,5,1


In [17]:
home_score, away_score, home_win, away_win = cpl.get_final_score_prediction(cpl_score_model,q1_roster_r,q2_roster_r,home_win,away_win)
print('home score',home_score,'home prob',home_win)
print('away score', away_score,'away prob', away_win)

home score 3 home prob 0.42
away score 2 away prob 0.25


In [18]:
hp,ap = cpl.get_classification(home_win, away_win,draw)
print(home_win, hp, home_score, away_win, ap, away_score)

0.42 3 3 0.25 1 2


In [19]:
score = home_score
team_pred = hp
q1_roster_c = cpl.add_classifier_features(q1_roster,score,team_pred)
q1_roster_c

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.71,0.71,0.74,0.9,0.77,0.5,0.52,0.45,0.68,0.42,...,3,4,4,4,4,2,4,5,4,1


In [20]:
score = away_score
team_pred = ap
q2_roster_c = cpl.add_classifier_features(q2_roster,score,team_pred)
q2_roster_c

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.79,0.92,0.75,0.37,0.33,0.67,0.67,0.35,0.37,0.73,...,3,4,4,4,4,2,4,4,5,1


In [21]:
new_cols1 = []
for counter in range(0,82):
    new_cols1.append(str(counter))

match_list = q1_roster_c.loc[0].tolist()
match_list.extend(q2_roster_c.loc[0].tolist())

home_array = pd.DataFrame([match_list],columns=new_cols1)

In [22]:
home_array

,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,81
0,0.71,0.71,0.74,0.9,0.77,0.5,0.52,0.45,0.68,0.42,...,3.0,4.0,4.0,4.0,4.0,2.0,4.0,4.0,5.0,1.0


In [23]:
#### loss prediction seems to be the greatest outcome
# mostly ignoring it and just dealing with the win and draw comparsion, because that will point to the loser

In [24]:
def get_final_game_prediction(model,home_array,home_score,away_score):
    
    def random_draw(x,y):
        if x > y:
            end = x
        else:
            end = y
        x = random.choice(range(end))
        y = x
        return x,y
    # get the model prediction probability of W, L, D
    
    ###### need to be sure of the order of these probabilities
    
    def roster_pred(model,array):
        prediction = model.predict([array.values[0]]).flatten()
        if prediction == 0:
            outcome = 'L'
        elif prediction == 1:
            outcome = 'D'
        else:
            outcome = 'W'
        return outcome
    
    def roster_prob(model,array):
        probability = model.predict_proba([array.values[0]]).flatten()
        return probability
    
    def norm(x,y,z):
        norm = x + y + z
        x, y, z = round(x / norm,2), round(y / norm,2), round(z / norm,2)
        return x, y, z
    
    
    # get the prediction from the classification model
    prediction = roster_pred(model,home_array)
    probability = roster_prob(model,home_array)
    #print(prediction,probability[0],probability[1],probability[2],home_score,away_score,'\n') # for debug leave it
    k.append([probability[0],probability[1],probability[2]])
    l.append([prediction,probability[0],probability[1],probability[2],home_score,away_score])
    p_l, p_d, p_w = probability[0],probability[1],probability[2]#norm(q1_w, q1_l, q1_d)
    
    #FIX THE SCORE ####
    # adjust score depending on the outcome from the prediction
    if prediction == 'W' and (home_score == away_score) or prediction == 'W' and (home_score < away_score):
        away_score = random.choice(range(home_score-1))
    if prediction == 'L' and (home_score == away_score) or prediction == 'L' and (home_score > away_score):
        home_score = random.choice(range(away_score-1))
    if prediction == 'D':
        home_score, away_score = random_draw(home_score, away_score)
    
    return p_w, p_l, p_d, home_score, away_score

In [25]:
def get_final_game_prediction(model,home_array,home_score,away_score):

    def random_draw(x,y):
        if x > y:
            end = x
        else:
            end = y
        x = random.choice(range(end))
        y = x
        return x,y
    # get the model prediction probability of W, L, D

    ###### need to be sure of the order of these probabilities

    def roster_pred(model,array):
        prediction = model.predict([array.values[0]]).flatten()
        if prediction == 0:
            outcome = 'L'
        elif prediction == 1:
            outcome = 'D'
        else:
            outcome = 'W'
        return outcome

    def roster_prob(model,array):
        probability = model.predict_proba([array.values[0]]).flatten()
        return probability

    def norm(x,y,z):
        norm = x + y + z
        x, y, z = round(x / norm,2), round(y / norm,2), round(z / norm,2)
        return x, y, z


    # get the prediction from the classification model
    prediction = roster_pred(model,home_array)
    probability = roster_prob(model,home_array)

    p_l, p_d, p_w = probability[0],probability[1],probability[2]#norm(q1_w, q1_l, q1_d)

    #FIX THE SCORE ####
    # adjust score depending on the outcome from the prediction
    if prediction == 'W' and (home_score == away_score) or prediction == 'W' and (home_score < away_score):
        away_score = random.choice(range(home_score-1))
    if prediction == 'L' and (home_score == away_score) or prediction == 'L' and (home_score > away_score):
        home_score = random.choice(range(away_score-1))
    if prediction == 'D':
        home_score, away_score = random_draw(home_score, away_score)
        
    k.append([probability[0],probability[1],probability[2]])
    l.append([prediction,probability[0],probability[1],probability[2],home_score,away_score])

    return p_w, p_l, p_d, home_score, away_score, prediction

In [9]:
k, l, ht, at = [],[],[],[]
for t in range(results.shape[0]):
    # home side
    q1 = schedule.iloc[t]['home']
    # away side
    q2 = schedule.iloc[t]['away']
    ht.append(q1)
    at.append(q2)
    compare = compare[compare['hr'] != 'E']
    t1_x, t1_y = cpl.get_NB_data(compare,q1)
    t2_x, t2_y = cpl.get_NB_data(compare,q2)
    
    game_info = schedule[schedule['home'] == q1]
    game_info = game_info[game_info['away'] == q2]
    game = game_info.iloc[0]['game']
    #game_h = cpl.get_home_away_comparison(stats,game,q1)
    #game_a = cpl.get_home_away_comparison(stats,game,q2)

    home_roster = cpl.best_roster(q1,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)
    away_roster = cpl.best_roster(q2,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)

    q1_roster = cpl.get_overall_roster(home_roster,player_info)
    q2_roster = cpl.get_overall_roster(away_roster,player_info)

    home_win, draw, away_win = cpl.get_nb_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
    q1_roster_r = cpl.add_regressor_features(q1_roster)
    q2_roster_r = cpl.add_regressor_features(q2_roster)
    home_score, away_score, home_win, away_win = cpl.get_final_score_prediction(cpl_score_model,q1_roster_r,q2_roster_r,home_win,away_win)
    hp,ap = cpl.get_classification(home_win, away_win,draw)
    score = home_score
    q1_roster_c = cpl.add_classifier_features(q1_roster,score,hp)
    score = away_score
    q2_roster_c = cpl.add_classifier_features(q2_roster,score,ap)
    
    new_cols1 = []
    for counter in range(0,82):
        new_cols1.append(str(counter))

    match_list = q1_roster_c.loc[0].tolist()
    match_list.extend(q2_roster_c.loc[0].tolist())

    home_array = pd.DataFrame([match_list],columns=new_cols1)
    
    home_win_new, away_win_new, draw_new, home_score, away_score, pred = cpl.get_final_game_prediction(cpl_classifier_model,home_array,home_score,away_score)
    print(f'\n{pred}-{q1} win prob {home_win_new} Score: {home_score} \n{q2} win prob {away_win_new} Score: {away_score} \ndraw prob {draw_new}')

CHECK SCORE 4 4 0.4 0.26
CHECK SCORE again 4 4 0.26 0.26
W 4 0 


W-Forge FC win prob 0.45 Score: 4 
Cavalry FC win prob 0.4 Score: 0 
draw prob 0.15
CHECK SCORE 3 2 0.33 0.33
CHECK SCORE again 3 2 0.33 0.33
L 0 2 


L-York9 FC win prob 0.4 Score: 0 
Atlético Ottawa win prob 0.4 Score: 2 
draw prob 0.2
CHECK SCORE 3 2 0.33 0.33
CHECK SCORE again 3 2 0.33 0.33
L 0 2 


L-HFX Wanderers FC win prob 0.35 Score: 0 
Pacific FC win prob 0.35 Score: 2 
draw prob 0.3
CHECK SCORE 3 4 0.33 0.33
CHECK SCORE again 3 4 0.33 0.33
W 3 0 


W-Valour FC win prob 0.45 Score: 3 
Cavalry FC win prob 0.3 Score: 0 
draw prob 0.25
CHECK SCORE 4 2 0.33 0.33
CHECK SCORE again 4 2 0.33 0.33
W 4 2 


W-Forge FC win prob 0.45 Score: 4 
FC Edmonton win prob 0.35 Score: 2 
draw prob 0.2
CHECK SCORE 2 3 0.33 0.33
CHECK SCORE again 2 3 0.33 0.33
L 2 3 


L-Pacific FC win prob 0.3 Score: 2 
York9 FC win prob 0.35 Score: 3 
draw prob 0.35
CHECK SCORE 2 3 0.33 0.33
CHECK SCORE again 2 3 0.33 0.33
L 2 3 


L-Atlético Otta

In [27]:
outcomes = pd.DataFrame(k,columns=['loss','draw','win'])
outcomes.describe()

,loss,draw,win
count,28.000000,28.000000,28.000000
mean,0.385714,0.282143,0.332143
std,0.084828,0.087363,0.100198
min,0.250000,0.100000,0.150000
25%,0.300000,0.237500,0.250000
50%,0.400000,0.300000,0.300000
75%,0.450000,0.350000,0.412500
max,0.550000,0.450000,0.500000


In [33]:
matches = pd.DataFrame(l,columns=['hr','l','d','w','hs','as'])
matches.insert(0,'home',ht)
matches['away'] = at
matches['ar'] = ['L' if x == 'W' else 'W' if x == 'L' else 'D' for x in matches['hr']]
matches
#"Even" if i%2==0 else "Odd" for i in range(8)

,home,hr,l,d,w,hs,as,away,ar
0,Forge FC,W,0.35,0.15,0.50,3,2,Cavalry FC,L
1,York9 FC,L,0.50,0.30,0.20,1,3,Atletico Ottawa,W
2,HFX Wanderers FC,L,0.50,0.25,0.25,0,2,Pacific FC,W
3,Valour FC,W,0.25,0.35,0.40,2,0,Cavalry FC,L
4,Forge FC,L,0.45,0.10,0.45,2,3,FC Edmonton,W
5,Pacific FC,D,0.25,0.45,0.30,0,0,York9 FC,D
6,Atletico Ottawa,W,0.30,0.25,0.45,3,2,Valour FC,L
7,HFX Wanderers FC,W,0.40,0.15,0.45,3,2,Forge FC,L
8,FC Edmonton,L,0.40,0.35,0.25,1,3,Cavalry FC,W
9,York9 FC,L,0.40,0.30,0.30,1,2,Valour FC,W


In [42]:
matches[matches['hr'] == 'W']['hr'].count(),matches[matches['hr'] == 'L']['hr'].count(),matches[matches['hr'] == 'D']['hr'].count()

(9, 15, 4)

In [44]:
total = matches[matches['hr'] == 'W']['hr'].count()+matches[matches['hr'] == 'L']['hr'].count()+matches[matches['hr'] == 'D']['hr'].count()
total

28

In [46]:
wins_h = matches[matches['hr'] == 'W']['hr'].count() / total
wins_h

0.32142857142857145

In [47]:
wins_a = matches[matches['hr'] == 'L']['hr'].count() / total
wins_a

0.5357142857142857

In [48]:
ha_d = matches[matches['hr'] == 'D']['hr'].count() / total
ha_d

0.14285714285714285